# Using music21 to parse MIDI data

In [1]:
from music21 import *

Let's start with a piece Ave Maria...

In [2]:
sBach = converter.parse("MIDI_Archive/Classical_Greatest/Bach/AveMaria.mid")
# sBach.show() #for this to work, you have to install a software called musescore

We can also play it!

In [3]:
sBach.show('midi')

# Extract MIDI sequence

To make a network, we want to extract the MIDI sequence for different parts. After piece is parsed, you can access different parts using piece.parts, and for each part p, p.notesAndRests would access the notes (or chords, aka notes happening together) and rests in the sequence, and secondsMap will provide the information of not only the note pitch, but also when it starts and when it ends.

However, there are rests that is not legit (those who didn't actually show in the sheet music) and notes with duration 0. We also probably want to keep track of very long rest, since that means a new sentence has begun. Therefore, we do a second round on the music piece, in order to filter notes that is shorter than a threhold and mark rest longer than a threshold.

In [4]:
def quantify_music(piece):
    from collections import defaultdict
    # first pass of the music, not removing notes and rests that is not reasonable
    midi_dict = {}
    time_dict = {}
    key = piece.analyze('key').name # tell us the key of the piece (eg. F major, C minor)
    duration_notes = []
    for i, p in enumerate(piece.parts):
        part_midi = []
        time = []
        for n in p.flat.notesAndRests.secondsMap: # secondsMap would include not only the note, but also the time information
            start = n['offsetSeconds']
            end = n['endTimeSeconds']
            time.append((start, end))
            element = n['element']
            try:
                part_midi.append(element.pitch.midi)
                if (end-start)!=0.0:
                    duration_notes.append((end-start))
            except:
                try:
                    part_midi.append([item.midi for item in element.pitches])
                except:
                    part_midi.append(128) # coding rest as 128
        midi_dict[i] = part_midi
        time_dict[i] = time
    # Indentify short rest and long rest
    midi_dict_prune = defaultdict(list)
    time_dict_prune = defaultdict(list)
    min_threshold = min(duration_notes)
    max_threshold = max(duration_notes)
#     print(min_threshold, max_threshold)
    for p in time_dict:
        for note, (start, end) in zip(midi_dict[p], time_dict[p]):
            if note!=128:
                midi_dict_prune[p].append(note)
                time_dict_prune[p].append((start, end))
            else:
                if end-start >= min_threshold:
                    if end-start > max_threshold:
                        midi_dict_prune[p].append(129)
                        time_dict_prune[p].append((start, end))
                    else:
                        midi_dict_prune[p].append(128)
                        time_dict_prune[p].append((start, end))
    return key, midi_dict_prune, time_dict_prune

In [5]:
sMozart = converter.parse("MIDI_Archive/Classical_Greatest/Mozart/K527 Overture ''Don Giovanni''.mid")

KeyboardInterrupt: 

In [6]:
%timeit key, midi_dict, time_dict = quantify_music(sMozart)

2.05 s ± 319 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
sBeatles = converter.parse("MIDI_Archive/Beatles/Beatles_Yesterday.mid")

In [11]:
key, midi_dict, time_dict = quantify_music(sBeatles)

In [12]:
len(midi_dict)

5

# Extract the whole corpus

## Get all the midi file name under this directory (including subdirectory)

In [7]:
import os
from glob import glob
import random
midis = []
start_dir1 = "MIDI_Archive/Classical_Greatest"
start_dir2 = "MIDI_Archive/Jazz"
start_dir3 = "MIDI_Archive/Metal_Rock"
start_dir4 = "MIDI_Archive/american_folk"

def walk_dir(start_dir):
    midis = []
    pattern   = "*.mid"
    for dir,_,_ in os.walk(start_dir):
        midis.extend(glob(os.path.join(dir,pattern))) 
    return midis

midis = walk_dir(start_dir1) + walk_dir(start_dir2) + walk_dir(start_dir3) + walk_dir(start_dir4)
# midis = random.sample(walk_dir(start_dir1), 50) + 
# random.sample(walk_dir(start_dir2), 50) + 
# random.sample(walk_dir(start_dir3), 50) + 
# random.sample(walk_dir(start_dir4), 50)

In [8]:
len(midis)

9843

In [9]:
import signal
import json
midi_corpus = {}
count = 0
ind = 0
failed_midi = []
big_midi = []
class TimeoutException(Exception):   # Custom exception class
    pass

def timeout_handler(signum, frame):   # Custom signal handler
    raise TimeoutException

# Change the behavior of SIGALRM
signal.signal(signal.SIGALRM, timeout_handler)

for midi in midis:
    # Start the timer. Once 10 seconds are over, a SIGALRM signal is sent.
    signal.alarm(10)    
    # This try/except loop ensures that 
    #   you'll catch TimeoutException when it's sent.
    try:
        try:
            piece = converter.parse(midi)
            key, midi_dict, duration_dict = quantify_music(piece)
            midi_corpus[midi.split("MIDI_Archive/")[1].split('.mid')[0].replace('/',' ')] = [key, midi_dict, duration_dict]
            count+=1
        except:
            print("failed:",midi)
            failed_midi.append(midi)
    except TimeoutException:
        print("big:",midi)
        big_midi.append(midi)
        signal.alarm(0)
        continue # continue the for loop if function A takes more than 5 second
    else:
        # Reset the alarm
        signal.alarm(0)
    if count > 200:        
        json.dump(midi_corpus, open("midi_corpus_%s.json"%ind,'w'),indent = 4)
        count = 0
        ind += 1

failed: MIDI_Archive/Classical_Greatest/Reinecke Piano Concerto n3 1mov.mid
failed: MIDI_Archive/Classical_Greatest/Buxethude Buxwv138 Prelude.mid
failed: MIDI_Archive/Classical_Greatest/Liszt Paganini Etude n3.mid
failed: MIDI_Archive/Classical_Greatest/Liszt Paganini Etude n2.mid
failed: MIDI_Archive/Classical_Greatest/Reinecke Piano Concerto n3 3mov.mid
failed: MIDI_Archive/Classical_Greatest/Tchaikovsky Lake Of The Swans Act 1 4mov.mid
failed: MIDI_Archive/Classical_Greatest/Liszt Paganini Etude n1.mid
failed: MIDI_Archive/Classical_Greatest/Bizet Symphony in C 1mov.mid
failed: MIDI_Archive/Classical_Greatest/Gershuin Rhapsody In Blue Piano Duet.mid
failed: MIDI_Archive/Classical_Greatest/Bizet Symphony in C 3mov.mid
failed: MIDI_Archive/Classical_Greatest/Dvorak Symphony op70 n7 4mov.mid
failed: MIDI_Archive/Classical_Greatest/Nicolai Overture The Merry Wives of Windsor.mid
failed: MIDI_Archive/Classical_Greatest/Dvorak Symphony op70 n7 1mov.mid
failed: MIDI_Archive/Classical_Grea

failed: MIDI_Archive/Classical_Greatest/Beethoven/Anhang 14-3.mid
failed: MIDI_Archive/Classical_Greatest/Beethoven/op051 3 Rondos.mid
failed: MIDI_Archive/Classical_Greatest/Beethoven/Symphonies/Symphony op93 n8 1mov.mid
failed: MIDI_Archive/Classical_Greatest/Beethoven/Symphonies/Symphony op68 n6 ''Pastorale''.mid
failed: MIDI_Archive/Classical_Greatest/Beethoven/Symphonies/Symphony op68 n6 1mov ''Pastorale''.mid
failed: MIDI_Archive/Classical_Greatest/Beethoven/Symphonies/Symphony n2 op36 1mov.mid
failed: MIDI_Archive/Classical_Greatest/Beethoven/Symphonies/Symphony n2 op36 4mov.mid
failed: MIDI_Archive/Classical_Greatest/Beethoven/Symphonies/Symphony n4 1mov.mid
failed: MIDI_Archive/Classical_Greatest/Beethoven/Symphonies/Symphony op68 n6 3mov ''Pastorale''.mid
failed: MIDI_Archive/Classical_Greatest/Beethoven/Symphonies/Symphony op125 n9 2mov ''Choral''.mid
failed: MIDI_Archive/Classical_Greatest/Beethoven/Symphonies/Symphony op125 n9 1mov ''Choral''.mid
failed: MIDI_Archive/Class

failed: MIDI_Archive/Classical_Greatest/Schumann/Symphony n4 op120 4mov.mid
failed: MIDI_Archive/Classical_Greatest/Schumann/Piano Sonata n2 op22 1mov.mid
failed: MIDI_Archive/Classical_Greatest/Schumann/Symphony n3 op97 ''Rhenish'' 1mov.mid
failed: MIDI_Archive/Classical_Greatest/Schumann/Schuman Toccata op7.mid
failed: MIDI_Archive/Classical_Greatest/Schumann/Symphony n3 op97 ''Rhenish'' 5mov.mid
failed: MIDI_Archive/Classical_Greatest/Schumann/Symphony n4 op120 2mov.mid
failed: MIDI_Archive/Classical_Greatest/Schumann/Papillons op2.mid
failed: MIDI_Archive/Classical_Greatest/Schumann/Fantasy in C op17.mid
failed: MIDI_Archive/Classical_Greatest/Schumann/Symphony n3 op97 ''Rhenish'' 2mov.mid
failed: MIDI_Archive/Classical_Greatest/Schumann/6 Impromptus ''Bilder aus Osten'' for 4-hands op66 n5.mid
failed: MIDI_Archive/Classical_Greatest/Schumann/Concertstück for Piano and Orchestra op92.mid
failed: MIDI_Archive/Classical_Greatest/Schumann/Album fur die Jugend op68 n14.mid
failed: MIDI

failed: MIDI_Archive/Classical_Greatest/Scarlatti/Sonata Longo375.mid
failed: MIDI_Archive/Classical_Greatest/Scarlatti/Sonata Longo465.mid
failed: MIDI_Archive/Classical_Greatest/Scarlatti/Sonata Longo25.mid
failed: MIDI_Archive/Classical_Greatest/Haendel/Harpsichord Suite n3 Hwv428 6mov.mid
failed: MIDI_Archive/Classical_Greatest/Haendel/Water Music Suite n2 Hwv349 2mov.mid
failed: MIDI_Archive/Classical_Greatest/Haendel/Harpsichord Suite in G Hwv432 6mov.mid
failed: MIDI_Archive/Classical_Greatest/Haendel/Organ Concerto op4 n4 3mov.mid
failed: MIDI_Archive/Classical_Greatest/Haendel/Royal Fireworks Suite 2mov Allegro.mid
failed: MIDI_Archive/Classical_Greatest/Haendel/Royal Fireworks Suite 1mov Overture.mid
failed: MIDI_Archive/Classical_Greatest/Haendel/Organ Concerto op4 n4 1mov.mid
failed: MIDI_Archive/Classical_Greatest/Haendel/Hwv435 Chaconne 21 Variations.mid
failed: MIDI_Archive/Classical_Greatest/Haendel/Harpsichord Suite Emin Jigg.mid
failed: MIDI_Archive/Classical_Greatest

failed: MIDI_Archive/Classical_Greatest/Bach/Bwv001- 400 Chorales/065600b_.mid
failed: MIDI_Archive/Classical_Greatest/Bach/Bwv001- 400 Chorales/014001b_.mid
failed: MIDI_Archive/Classical_Greatest/Bach/Bwv001- 400 Chorales/Bwv001.mid
failed: MIDI_Archive/Classical_Greatest/Bach/Bwv1080 The Art Of Fugue/The Art of the Fugue BWV1080.mid
failed: MIDI_Archive/Classical_Greatest/Bach/Bwv ''Little Notebook for Anna Magdalena Bach''/07 Rondo.mid
failed: MIDI_Archive/Classical_Greatest/Bach/Bwv988 The Goldberg Variations/Variation 13.mid
failed: MIDI_Archive/Classical_Greatest/Bach/Harpsichord Concertos/Bwv1054 Harpsichord Concerto n3 3mov.mid
failed: MIDI_Archive/Classical_Greatest/Bach/Harpsichord Concertos/Bwv1064 Three Harpsichords Concerto n2 3mov.mid
failed: MIDI_Archive/Classical_Greatest/Bach/Harpsichord Concertos/Bwv1064 Three Harpsichords Concerto n2 1mov.mid
failed: MIDI_Archive/Classical_Greatest/Bach/Harpsichord Concertos/Bwv1054 Harpsichord Concerto n3 1mov.mid
failed: MIDI_Arch

...

failed: MIDI_Archive/Metal_Rock/midi/b/black_sabbath/Sabbath_Bloody_Sabbath_2.mid
failed: MIDI_Archive/Metal_Rock/midi/b/blink182/M_Ms.mid
failed: MIDI_Archive/Metal_Rock/midi/b/blink182/Rollercoaster.mid
failed: MIDI_Archive/Metal_Rock/midi/b/blink182/Adams_Song.mid
failed: MIDI_Archive/Metal_Rock/midi/b/blink182/Always.mid
failed: MIDI_Archive/Metal_Rock/midi/b/blink182/Aliens_Exist.mid
failed: MIDI_Archive/Metal_Rock/midi/k/korn/Blind_4.mid
failed: MIDI_Archive/Metal_Rock/midi/k/korn/Blind_2.mid
failed: MIDI_Archive/Metal_Rock/midi/k/korn/Lets_Get_This_Party_Started_2.mid
failed: MIDI_Archive/Metal_Rock/midi/k/korn/Blind_1.mid
failed: MIDI_Archive/Metal_Rock/midi/k/korn/Ass_Itch_3.mid
failed: MIDI_Archive/Metal_Rock/midi/k/korn/Ass_Itch_1.mid
failed: MIDI_Archive/Metal_Rock/midi/k/korn/Clown_1.mid
failed: MIDI_Archive/Metal_Rock/midi/k/korn/Good_God_2.mid
failed: MIDI_Archive/Metal_Rock/midi/l/led_zeppelin/Ten_Years_Gone.mid
failed: MIDI_Archive/Metal_Rock/midi/l/led_zeppelin/Dazed_

failed: MIDI_Archive/american_folk/webster/jpw62oahgadib.mid
failed: MIDI_Archive/american_folk/webster/jpw59wl-apw67.mid
failed: MIDI_Archive/american_folk/webster/jpw65bita.mid
failed: MIDI_Archive/american_folk/webster/jpw62watgahs.mid
failed: MIDI_Archive/american_folk/webster/jpw66tgr.mid
failed: MIDI_Archive/american_folk/webster/jpw57lorena.mid
failed: MIDI_Archive/american_folk/webster/jpw66ptf.mid
failed: MIDI_Archive/american_folk/webster/jpw62tnes.mid
failed: MIDI_Archive/american_folk/webster/jpw62tgr.mid
failed: MIDI_Archive/american_folk/webster/jpw74utbs.mid
failed: MIDI_Archive/american_folk/webster/jpw73dih.mid
failed: MIDI_Archive/american_folk/webster/jpw55toc.mid
failed: MIDI_Archive/american_folk/webster/jpw68hfgg.mid
failed: MIDI_Archive/american_folk/webster/jpw62futrb.mid
failed: MIDI_Archive/american_folk/webster/jpw61talatwfm.mid
failed: MIDI_Archive/american_folk/heinrich/aph23tc.mid
failed: MIDI_Archive/american_folk/heinrich/aph25tmvfp.mid
failed: MIDI_Arch

failed: MIDI_Archive/american_folk/1800s/43dnyg.mid
failed: MIDI_Archive/american_folk/1800s/84wafwhijv.mid
failed: MIDI_Archive/american_folk/1800s/80man.mid
failed: MIDI_Archive/american_folk/1800s/36ootc.mid
failed: MIDI_Archive/american_folk/1800s/46sotsn-hk.mid
failed: MIDI_Archive/american_folk/1800s/60ttbdokh.mid
failed: MIDI_Archive/american_folk/1800s/76ttiwwab.mid
failed: MIDI_Archive/american_folk/1800s/47sda.mid
failed: MIDI_Archive/american_folk/1800s/54fdotua.mid
failed: MIDI_Archive/american_folk/1800s/35tst.mid
failed: MIDI_Archive/american_folk/1800s/67teq.mid
failed: MIDI_Archive/american_folk/1800s/40jb.mid
failed: MIDI_Archive/american_folk/1800s/66gp.mid
failed: MIDI_Archive/american_folk/1800s/74othttph.mid
failed: MIDI_Archive/american_folk/1800s/84tfitp.mid
failed: MIDI_Archive/american_folk/1800s/76cc.mid
failed: MIDI_Archive/american_folk/1800s/50gpdc.mid
failed: MIDI_Archive/american_folk/1800s/79ightc.mid
failed: MIDI_Archive/american_folk/1800s/78trusting.m

failed: MIDI_Archive/american_folk/1800s/98cw.mid
failed: MIDI_Archive/american_folk/1800s/53tosoj.mid
failed: MIDI_Archive/american_folk/1800s/92tb.mid
failed: MIDI_Archive/american_folk/1800s/50iofc.mid
failed: MIDI_Archive/american_folk/1800s/71rcgm.mid
failed: MIDI_Archive/american_folk/1800s/59lmkhfhm.mid
failed: MIDI_Archive/american_folk/1800s/45tfoig.mid
failed: MIDI_Archive/american_folk/1800s/52rrybotolh.mid
failed: MIDI_Archive/american_folk/1800s/94phoebe.mid
failed: MIDI_Archive/american_folk/1800s/50tbs.mid
failed: MIDI_Archive/american_folk/1800s/53tdwolk.mid
failed: MIDI_Archive/american_folk/1800s/66tbom.mid
failed: MIDI_Archive/american_folk/1800s/52tyfah.mid
failed: MIDI_Archive/american_folk/1800s/54dtrdto.mid
failed: MIDI_Archive/american_folk/1800s/47botr.mid
failed: MIDI_Archive/american_folk/1800s/43tss.mid
failed: MIDI_Archive/american_folk/1800s/76ch.mid
failed: MIDI_Archive/american_folk/1800s/51wmnm.mid
failed: MIDI_Archive/american_folk/1800s/50tob.mid
fail

failed: MIDI_Archive/american_folk/civilwar2/62gaonl.mid
failed: MIDI_Archive/american_folk/civilwar2/62tlmwahld.mid
failed: MIDI_Archive/american_folk/civilwar2/61stof.mid
failed: MIDI_Archive/american_folk/civilwar2/61ahttfof.mid
failed: MIDI_Archive/american_folk/civilwar2/61al.mid
failed: MIDI_Archive/american_folk/civilwar2/61ightd.mid
failed: MIDI_Archive/american_folk/civilwar2/62tsoatuta.mid
failed: MIDI_Archive/american_folk/civilwar2/63bmioln.mid
failed: MIDI_Archive/american_folk/civilwar2/63attny.mid
failed: MIDI_Archive/american_folk/civilwar2/62brfs.mid
failed: MIDI_Archive/american_folk/thomas/jrt58mdgb.mid
failed: MIDI_Archive/american_folk/thomas/jrt60taol.mid
failed: MIDI_Archive/american_folk/thomas/jrt53gb.mid
failed: MIDI_Archive/american_folk/thomas/jrt79snm.mid
failed: MIDI_Archive/american_folk/thomas/jrt61mgpc.mid
failed: MIDI_Archive/american_folk/thomas/jrt74thtrtw.mid
failed: MIDI_Archive/american_folk/thomas/jrt74ows.mid
failed: MIDI_Archive/american_folk/t

In [10]:
len(midi_corpus)

8090

# Simple network generation (one order)

In [11]:
import pickle
for each_piece in midi_corpus:
    from collections import Counter
    for each_part in midi_corpus[each_piece][1]:
        edges = []
        sequence = midi_corpus[each_piece][1][each_part]
        sequence = [item if type(item)!=list else max(item) for item in sequence]
        edges = list(zip(sequence[:-1], sequence[1:]))
        edges_with_weight = dict(Counter(edges))
        pickle.dump(edges_with_weight, open("Simple Network/%s %s.pickle"%(each_piece.replace('/',' '), each_part), "wb"))